# **SUPPORING LIBRARIES**

In [9]:
import pandas as pd
import re
import math
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import preprocessing
import numpy as np
import pandas as pd
import re as re
import lxml.html
import lxml.html.clean
from nltk.tokenize import RegexpTokenizer
from string import digits
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
import scipy
from scipy.sparse import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree

nltk.download('words')
nltk.download('stopwords')


token = RegexpTokenizer("[\w']+")
sw = stopwords.words('english')
tokenizer = (nltk.corpus.words.words())

nltk.download('words')
nltk.download('stopwords')
from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **DATA FETCHING FROM DRIVE**

In [11]:

token = RegexpTokenizer("[\w']+")
sw = stopwords.words('english')
tokenizer = (nltk.corpus.words.words())

#reading training data from csv files
train_data = pd.read_csv('/content/drive/My Drive/dataset/training/data_train.csv', names=['country', 'sku_id', 'title', 'category_lvl_1', 'category_lvl_2', 'category_lvl_3', 'short_description', 'price', 'product_type'], encoding = 'utf8')
train_clarity_labels = pd.read_csv('/content/drive/My Drive/dataset/training/clarity_train.csv', names=['clarity'], encoding = 'utf8')
train_conciseness_labels = pd.read_csv('/content/drive/My Drive/dataset/training/conciseness_train.csv', names=['conciseness'], encoding = 'utf8')
train_data['clarity'] = train_clarity_labels
train_data['conciseness'] = train_conciseness_labels



# **DATA SCLICING FOR TRAINING AND TESING**

In [ ]:
# train_data.shape
training = train_data[:25400][:]
training


In [ ]:
testing = train_data[25401:][:]
testing

# **DATA CLEANING FUNCTIONS**

In [14]:

# preprocessing --
       

train_data['category_lvl_3'] = train_data['category_lvl_3'].fillna('NA')
#null values check
def null_value_chk(data):
    for column in data.columns:
        print(column, data[column].isnull().sum())


#html tags removing from description
def remove_tags(string):
    
    doc = lxml.html.fromstring(string)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    text = doc.text_content()
    return text

def clear_description(data):
    new_string = re.sub('[:!@#$]','',data )
    return new_string

#data cleaning of null vales of category

def category_data(data):
    for idx in range(0 , len(data)):
        if(pd.isnull(data['category_lvl_3'][idx]) ):
            data.loc[idx, 'category_lvl_3'] = data.loc[idx]['category_lvl_1'] + " " + data.loc[idx]['category_lvl_2']


def clean_data(data):
    
    #train_data['short_description'] = train_data['short_description'].apply(lambda x: 'no description')
    #html tags removal
    data['short_description'] = data['short_description'].apply(lambda cw : remove_tags(str(cw)))
    data['short_description'] = data['short_description'].apply(lambda x:clear_description(str(x)))

    return data



# **TRAINING DATA CLEANING**

In [15]:
training=clean_data(training)
training.iloc[2:6]


,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type,clarity,conciseness
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,"150cm mini microphone compatible for iPhone, ...",25.07,international,1,0
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,ANMYNA Complaint Silky Set (Shampoo 520ml + C...,118.00,local,1,1
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,100% Authentic Rrefresh and brighten skin Ant...,114.80,international,1,1
5,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,Genuine Windows® 8.1 13.3in 169 HD TOUCH In...,2599.00,local,1,1


# **TESTING DATA CLEANING**

In [16]:
testing=clean_data(testing)
testing.iloc[2:6]


,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type,clarity,conciseness
25403,my,OE702ELAAARJX1ANMY,LD OTG USB Flash Drive 64GB Pen Drive USB 2.0 ...,Computers & Laptops,Storage,OTG Drives,Timeto market2016 ProductsStatusStock StyleLa...,96.57,international,1,0
25404,my,OE702ELAACACLAANMY,Hard Armor Protect Back Cover Case For Xiaomi ...,Mobiles & Tablets,Accessories,Phone Cases,Made of high quality TPU Slim and lightweight...,25.00,international,1,1
25405,my,OE702ELAACL7F2ANMY,Phone case for iPhone 5/5s/SE Fox Pushed The H...,Mobiles & Tablets,Accessories,Phone Cases,High Quality phone Case for iPhone 5/5s/SE Fa...,19.90,international,1,0
25406,my,OE702ELAACNY5AANMY,Pop Cartridges for Canon CL741 (Color),Computers & Laptops,Printers & Accessories,Ink,Simply pop the top off your original Canon ca...,35.00,local,1,1


# **FEATURES GENERATION OF TRAINING AND TESTING DATA**

In [20]:
colors_df = set()
brands_df = set()

def __load__():
    global colors_df
    global brands_df
    
    colors_df = set(line.strip().lower() for line in open('/content/drive/My Drive/dataset/colors.txt',"r", encoding='ISO-8859-1'))
    brands_df = set(line.strip().lower() for line in open('/content/drive/My Drive/dataset/lazada_brands.txt',"r", encoding='ISO-8859-1'))

__load__()

def jaccard_sim(s1, s2):
    inter = float(len(s1&s2))
    union = float(len(s1|s2))
    return inter / union  if union > 0 else -1.0

def has_numeric_value(word):
    return bool(re.search(r'\d', word))

def special_char_count(text):
    special_chars = '''!"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~ '''
    return sum(1 for c in text if c in special_chars)

def price_country(price, country):
    
    if country == 'my':
        price *= 0.32
    elif country != 'sg':
        price *= 0.074
    
    return price

def is_color(color):
    global colors_df
    return color.lower() in colors_df

def is_brand(brand):
    global brands_df
    return brand in brands_df


def generate_surface_features(df):
    
    feat_names = ['title_words_count', 'title_unique_words_count', 'title_duplicate_words_count', 'title_duplicate_words_ratio',\
            'title_avg_word_length','title_sku','title_has_numeric_value','title_desc_sim', 'title_cat1_sim',\
            'title_cat2_sim', 'title_cat3_sim', 'title_special_chars_count', 'price_c','title_has_color', 'title_has_brand']
   
    feats_data = []
    
    for i, row in df.iterrows():
        
        #new features
        feat = [] 
        
        title = row['title'].lower()
        desc = row['short_description']
        sku_id = row['sku_id'].lower()
        price = float(row['price'])
        country = row['country']
        
        title_words = title.split(' ')
        title_words_count = len(title_words)
        feat.append(title_words_count)
            
        desc_words = desc.split(' ')
            
            
        #if there are many duplicate words, the title will be not concise
    
        title_unique_words = [w for w in set(title_words)]
        
        title_unique_words_count = len(title_unique_words)
        feat.append(title_unique_words_count)
        
        title_duplicate_words_count = title_words_count - title_unique_words_count
        feat.append(title_duplicate_words_count)
            
        title_duplicate_words_ratio = title_duplicate_words_count / title_words_count
        feat.append(title_duplicate_words_ratio)
        
        title_avg_word_length = len(title)/title_words_count
        feat.append(title_avg_word_length)
               
        
        # if first two characters matches with sku_id. first word in title usually shows brand
        title_sku = int(title[:2] == sku_id[:2])
        feat.append(title_sku)
            
        #contains some numeric value which may show product model
        title_has_numeric_value = int(any([ has_numeric_value(w) for w in title_words ]))
        feat.append(title_has_numeric_value)
        
        #jaccard similarity score between title and description
        title_desc_sim = jaccard_sim(set(title_words),set(desc_words))
        feat.append(title_desc_sim)
        
        
        
        cat1 = row['category_lvl_1'].lower().split()
        cat2 = row['category_lvl_2'].lower().split()
        cat3 = row['category_lvl_3'].lower().split()
        
        # similarity between title words and category words
        feat.append(len(set(title_words)&set(cat1)))
        feat.append(len(set(title_words)&set(cat2)))
        feat.append(len(set(title_words)&set(cat3)))
        
        
        #special character count
        title_special_chars_count = special_char_count(title)
        feat.append(title_special_chars_count)
        
        #price country
        price_c = price_country(price,country)
        feat.append(price_c) 
        
        #title contains color
        title_has_color = int(any([is_color(w) for w in title_words]))
        feat.append(title_has_color)
        
        #title has brand
        title_has_brand = int(any([is_brand(w) for w in title_words]))
        feat.append(title_has_brand)
        
        feats_data.append(feat)
        
    
    return pd.DataFrame(feats_data, columns=feat_names)
    #return scipy.sparse.csr_matrix(feats_data)
        
train_data_feats = generate_surface_features(training)
testing_data_feats = generate_surface_features(testing)

In [21]:
train_data_feats

,title_words_count,title_unique_words_count,title_duplicate_words_count,title_duplicate_words_ratio,title_avg_word_length,title_sku,title_has_numeric_value,title_desc_sim,title_cat1_sim,title_cat2_sim,title_cat3_sim,title_special_chars_count,price_c,title_has_color,title_has_brand
0,8,8,0,0.000000,5.500000,1,0,0.000000,0,0,0,7,15.6800,1,1
1,7,7,0,0.000000,7.000000,0,1,0.000000,0,0,0,8,40.9600,0,0
2,19,19,0,0.000000,7.263158,1,1,0.103448,0,0,0,20,8.0224,0,1
3,10,10,0,0.000000,6.900000,1,1,0.107143,0,0,0,12,37.7600,1,0
4,9,9,0,0.000000,5.777778,1,1,0.052632,0,0,2,8,36.7360,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25395,9,9,0,0.000000,7.000000,0,1,0.133333,0,0,0,12,5.7280,0,1
25396,13,13,0,0.000000,7.307692,0,1,0.000000,0,0,0,17,11.4432,1,1
25397,15,15,0,0.000000,7.933333,0,0,0.090909,0,0,1,16,27.9680,0,1
25398,11,10,1,0.090909,7.000000,0,0,0.000000,0,0,0,10,6.6560,0,1


In [22]:
testing_data_feats

,title_words_count,title_unique_words_count,title_duplicate_words_count,title_duplicate_words_ratio,title_avg_word_length,title_sku,title_has_numeric_value,title_desc_sim,title_cat1_sim,title_cat2_sim,title_cat3_sim,title_special_chars_count,price_c,title_has_color,title_has_brand
0,11,11,0,0.000000,6.454545,0,1,0.200000,0,0,1,13,11.5200,0,1
1,20,19,1,0.050000,6.350000,0,1,0.068966,0,0,0,25,13.1200,0,1
2,17,15,2,0.117647,6.058824,0,1,0.041667,0,0,1,21,30.9024,0,1
3,11,11,0,0.000000,5.454545,0,0,0.000000,0,0,0,12,8.0000,0,1
4,18,15,3,0.166667,5.277778,0,1,0.071429,0,0,1,19,6.3680,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10877,14,14,0,0.000000,7.071429,1,1,0.022556,0,0,0,18,67.0900,1,1
10878,6,6,0,0.000000,5.166667,1,1,0.000000,0,0,0,5,69.0000,0,1
10879,25,23,2,0.080000,5.480000,0,1,0.036697,0,0,0,29,35.0000,0,1
10880,5,5,0,0.000000,7.800000,1,0,0.000000,0,0,0,7,23.0000,0,1


In [24]:
def generate_title_text_features(train_data,validation_data,analyzer="char", n_grams=(2,6), min_df = 1, max_df = 1.0):
    #title text features ngrams
    
    n_tr = train_data.shape[0]
    
    
    df = pd.concat((train_data,validation_data))
    
    countVectorizer = CountVectorizer(analyzer=analyzer, ngram_range=n_grams, min_df=min_df, max_df = max_df, lowercase=True)
    countVectorizer.fit(df['title'])
    X = countVectorizer.transform(df['title'].values)
    
    X_tr = X[:n_tr,:]
    X_val = X[n_tr:,:]
    
    #print(countVectorizer.get_feature_names())
    with open("title_ng_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("title_ng_val.pickle","wb") as f:
        pickle.dump(X_val, f)
        
    return X_tr,X_val
  
X_title_count_tr,X_title_count_val = generate_title_text_features(training,testing, min_df = 0.005)
# train_data_feats['title_text_count_feat'] = generate_title_text_features(train_data)
X_title_count_tr.shape

(25400, 8774)

In [27]:
def extract_brand_features(train_data, validation_data, analyzer='word', ngram_range=(1, 1), lowercase=True):
    
    n_trains = train_data.shape[0]
    
    df = pd.concat((train_data,validation_data))
    
    brands = [x.strip() for x in open("/content/drive/My Drive/dataset/lazada_brands.txt","r", encoding='ISO-8859-1').readlines()]
    vect = TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, decode_error='replace')
    vect.fit(brands)
    X = vect.transform(df["title"].map(lambda x: x.replace('(', ' (').replace(')', ' )').lower()).values)
    X_tr = X[:n_trains,:]
    X_val = X[n_trains:,:]
    feat_train_file = "title_brands.trn"
    feat_test_file = "title_brands.tst"

    with open("brands_feats_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("brands_feats_val.pickle","wb") as f:
        pickle.dump(X_val, f)

extract_brand_features(training,testing)

In [28]:
def extract_color_features(train_data, validation_data, analyzer='word', ngram_range=(1, 1), lowercase=True):
    
    n_trains = train_data.shape[0]
    
    df = pd.concat((train_data,validation_data))
    
    colors = [x.strip() for x in open("/content/drive/My Drive/dataset/colors.txt","r", encoding='ISO-8859-1').readlines()]
    
    vect = TfidfVectorizer(analyzer=analyzer, ngram_range=ngram_range, decode_error='replace')
    vect.fit(colors)
    
    X = vect.transform(df["title"].map(lambda x: x.replace('(', ' (').replace(')', ' )').lower()).values)
    X_tr = X[:n_trains,:]
    X_val = X[n_trains:,:]
    feat_train_file = "title_color.trn"
    feat_test_file = "title_color.tst"

    with open("color_feats_tr.pickle","wb") as f:
        pickle.dump(X_tr, f)
    
    with open("color_feats_val.pickle","wb") as f:
        pickle.dump(X_val, f)

extract_color_features(training,testing)

In [50]:
with open("/content/drive/My Drive/dataset/title_ng_tr.pickle","rb") as f:
    X_title_count_tr = pickle.load(f)
    X_title_count_tr = X_title_count_tr [:25400][:]
    

    X_title_count_val = X_title_count_tr[25401:][:]

with open("/content/drive/My Drive/dataset/color_feats_tr.pickle","rb") as f:
    X_title_color_tr = pickle.load(f)
    X_title_color_tr = X_title_color_tr[:25400][:]

    
    X_title_color_val = X_title_color_tr[25401:][:]

with open("/content/drive/My Drive/dataset/brands_feats_tr.pickle","rb") as f:
    X_title_brands_tr = pickle.load(f)
    X_title_brands_tr = X_title_brands_tr[:25400][:]
    

    X_title_brands_val = X_title_brands_tr[25401:][:]

# **SPARSE MATRIX OF TRAINING DATA FEATURES**

In [51]:
X_surf_feats_tr = scipy.sparse.csr_matrix(train_data_feats)

X_feats_tr = hstack((X_title_count_tr,X_title_color_tr,X_title_brands_tr, X_surf_feats_tr))

X_feats_tr = X_feats_tr.tocsr()

X_feats_tr


<25400x28731 sparse matrix of type '<class 'numpy.float64'>'
	with 5536865 stored elements in Compressed Sparse Row format>

# **SPARSE MATRIX OF TESTING DATA FEATURES**

In [53]:
X_surf_feats_val = scipy.sparse.csr_matrix(testing_data_feats)
X_surf_feats_val

X_feats_val = hstack((X_title_count_val,X_title_color_val,X_title_brands_val, X_surf_feats_val))

X_feats_val = X_feats_val.tocsr()

X_feats_val

<10882x28731 sparse matrix of type '<class 'numpy.float64'>'
	with 94555 stored elements in Compressed Sparse Row format>

In [55]:
X_title_count_tr = X_title_count_tr.astype('float32')
X_title_count_tr



<25400x8768 sparse matrix of type '<class 'numpy.float32'>'
	with 5085973 stored elements in Compressed Sparse Row format>

# **LGB Conciseness**

In [57]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
lgb_model_concise = lgb.LGBMClassifier()
YStrat = training['conciseness'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['conciseness'][train_index]
    test_X = X[test_index]
    test_Y = training['conciseness'][test_index]
    lgb_model_concise.fit(train_X, train_Y)
    predictions = lgb_model_concise.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("LGB model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("LGB model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)

LGB model accuracy score for K fold 1 = 0.8303149606299213
LGB model rmse score for K fold 1 = 0.4119284396228048

LGB model accuracy score for K fold 2 = 0.8460629921259842
LGB model rmse score for K fold 2 = 0.39234806979774445

LGB model accuracy score for K fold 3 = 0.8488188976377953
LGB model rmse score for K fold 3 = 0.38882014140500065

LGB model accuracy score for K fold 4 = 0.8622047244094488
LGB model rmse score for K fold 4 = 0.3712078603566352

LGB model accuracy score for K fold 5 = 0.8590551181102363
LGB model rmse score for K fold 5 = 0.3754262669150412

LGB model accuracy score for K fold 6 = 0.85
LGB model rmse score for K fold 6 = 0.3872983346207417

LGB model accuracy score for K fold 7 = 0.8531496062992125
LGB model rmse score for K fold 7 = 0.3832106388147221

LGB model accuracy score for K fold 8 = 0.8472440944881889
LGB model rmse score for K fold 8 = 0.3908399998871802

LGB model accuracy score for K fold 9 = 0.844488188976378
LGB model rmse score for K fold 9 

# **LGB Clarity**

In [58]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
lgb_model_clar = lgb.LGBMClassifier()
YStrat = training['clarity'].values
step = 1
X = X_feats_tr

for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['clarity'][train_index]
    test_X = X[test_index]
    test_Y = training['clarity'][test_index]
    lgb_model_clar.fit(train_X, train_Y)
    predictions = lgb_model_clar.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    rmse = math.sqrt(mean_squared_error(test_Y, predictions))
    print("LGB model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("LGB model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

LGB model accuracy score for K fold 1 = 0.9362204724409449
LGB model rmse score for K fold 1 = 0.2525460899698412

LGB model accuracy score for K fold 2 = 0.9421259842519685
LGB model rmse score for K fold 2 = 0.24057018881821476

LGB model accuracy score for K fold 3 = 0.9425196850393701
LGB model rmse score for K fold 3 = 0.23975052650751347

LGB model accuracy score for K fold 4 = 0.9433070866141732
LGB model rmse score for K fold 4 = 0.2381027370397635

LGB model accuracy score for K fold 5 = 0.9480314960629921
LGB model rmse score for K fold 5 = 0.2279660148728487

LGB model accuracy score for K fold 6 = 0.9417322834645669
LGB model rmse score for K fold 6 = 0.2413870678711539

LGB model accuracy score for K fold 7 = 0.9437007874015748
LGB model rmse score for K fold 7 = 0.23727455109729995

LGB model accuracy score for K fold 8 = 0.9405511811023622
LGB model rmse score for K fold 8 = 0.24382128475101963

LGB model accuracy score for K fold 9 = 0.9401574803149606
LGB model rmse sc

# **LGB PRIDICTION (CLARITY) ON TESTING DATA**

In [62]:
    predictions = lgb_model_clar.predict(X_feats_val)
    accuracy = accuracy_score(testing['clarity'], predictions)
    print("ACCURACY ON TESTING DATA VIA LGB FOR CLARITY",accuracy)


ACCURACY ON TESTING DATA VIA LGB FOR CLARITY 0.94376033817313


# **LGB PRIDICTION (CONCISENESS) ON TESTING DATA**

In [64]:
    predictions = lgb_model_concise.predict(X_feats_val)
    accuracy = accuracy_score(testing['conciseness'], predictions)
    print("ACCURACY ON TESTING DATA VIA LGB FOR CONCISENESS",accuracy)


ACCURACY ON TESTING DATA VIA LGB FOR CONCISENESS 0.7424186730380444


# **Decision Tree Conciseness**

In [65]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
dt_concise = tree.DecisionTreeClassifier()
YStrat = training['conciseness'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['conciseness'][train_index]
    test_X = X[test_index]
    test_Y = training['conciseness'][test_index]
    dt_concise.fit(train_X, train_Y)
    predictions = dt_concise.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("DT model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("DT model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)


DT model accuracy score for K fold 1 = 0.7586614173228347
DT model rmse score for K fold 1 = 0.49126223412467335

DT model accuracy score for K fold 2 = 0.7688976377952756
DT model rmse score for K fold 2 = 0.4807310705630794

DT model accuracy score for K fold 3 = 0.7519685039370079
DT model rmse score for K fold 3 = 0.4980276057238114

DT model accuracy score for K fold 4 = 0.7484251968503937
DT model rmse score for K fold 4 = 0.5015723309250684

DT model accuracy score for K fold 5 = 0.765748031496063
DT model rmse score for K fold 5 = 0.48399583521342104

DT model accuracy score for K fold 6 = 0.7511811023622047
DT model rmse score for K fold 6 = 0.4988174993299606

DT model accuracy score for K fold 7 = 0.7566929133858268
DT model rmse score for K fold 7 = 0.4932616816803969

DT model accuracy score for K fold 8 = 0.7665354330708661
DT model rmse score for K fold 8 = 0.48318171212198613

DT model accuracy score for K fold 9 = 0.7641732283464567
DT model rmse score for K fold 9 = 0

# **Decision Tree Clarity**

In [66]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
dt_clar = tree.DecisionTreeClassifier()
YStrat = training['clarity'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['clarity'][train_index]
    test_X = X[test_index]
    test_Y = training['clarity'][test_index]
    dt_clar.fit(train_X, train_Y)
    predictions = dt_clar.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("DT model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("DT model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)

DT model accuracy score for K fold 1 = 0.9204724409448819
DT model rmse score for K fold 1 = 0.28200631031081225

DT model accuracy score for K fold 2 = 0.9106299212598425
DT model rmse score for K fold 2 = 0.29894828773578463

DT model accuracy score for K fold 3 = 0.9153543307086615
DT model rmse score for K fold 3 = 0.29093928798176877

DT model accuracy score for K fold 4 = 0.9208661417322834
DT model rmse score for K fold 4 = 0.2813074088390075

DT model accuracy score for K fold 5 = 0.9173228346456693
DT model rmse score for K fold 5 = 0.2875363722285073

DT model accuracy score for K fold 6 = 0.9094488188976378
DT model rmse score for K fold 6 = 0.3009172329767144

DT model accuracy score for K fold 7 = 0.9216535433070866
DT model rmse score for K fold 7 = 0.27990437062131307

DT model accuracy score for K fold 8 = 0.9118110236220472
DT model rmse score for K fold 8 = 0.29696628828530813

DT model accuracy score for K fold 9 = 0.9051181102362205
DT model rmse score for K fold 9 

# **DECISION TREE PRIDICTION CONCISENESS**

In [71]:
    predictions = dt_concise.predict(X_feats_val)
    accuracy = accuracy_score(testing['conciseness'], predictions)
    print("ACCURACY ON TESTING DATA VIA DT FOR CONCISENESS",accuracy)


ACCURACY ON TESTING DATA VIA DT FOR CONCISENESS 0.770170924462415


# **DECISION TREE PRIDICTION CLARITY**

In [68]:
    predictions = dt_clar.predict(X_feats_val)
    accuracy = accuracy_score(testing['clarity'], predictions)
    print("ACCURACY ON TESTING DATA VIA DT FOR CLARITY",accuracy)


ACCURACY ON TESTING DATA VIA DT FOR CLARITY 0.9436684433008639


# **XTREME GRADIENT BOOSTING CONCISENESS**

In [69]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
xgb_concise = GradientBoostingClassifier()
YStrat = training['conciseness'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['conciseness'][train_index]
    test_X = X[test_index]
    test_Y = training['conciseness'][test_index]
    xgb_concise.fit(train_X, train_Y)
    predictions = xgb_concise.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("xgb model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("xgb model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)

xgb model accuracy score for K fold 1 = 0.8137795275590551
xgb model rmse score for K fold 1 = 0.4315327014734166

xgb model accuracy score for K fold 2 = 0.8468503937007874
xgb model rmse score for K fold 2 = 0.3913433355753137

xgb model accuracy score for K fold 3 = 0.8322834645669291
xgb model rmse score for K fold 3 = 0.4095320932882683

xgb model accuracy score for K fold 4 = 0.8480314960629921
xgb model rmse score for K fold 4 = 0.3898313788511744

xgb model accuracy score for K fold 5 = 0.8318897637795276
xgb model rmse score for K fold 5 = 0.41001248300566706

xgb model accuracy score for K fold 6 = 0.8401574803149606
xgb model rmse score for K fold 6 = 0.3998031011448503

xgb model accuracy score for K fold 7 = 0.8208661417322834
xgb model rmse score for K fold 7 = 0.423242079982268

xgb model accuracy score for K fold 8 = 0.8236220472440945
xgb model rmse score for K fold 8 = 0.4199737524606812

xgb model accuracy score for K fold 9 = 0.8338582677165355
xgb model rmse score 

# **XTREME GRADIENT BOOSTING CLARITY**

In [70]:
k = 10
kf = StratifiedKFold(n_splits=k, shuffle=True)
xgb_clar = GradientBoostingClassifier()
YStrat = training['clarity'].values
step = 1
X = X_feats_tr
avg_rmse = 0
for train_index, test_index in kf.split(X,YStrat): 
    train_X = X[train_index]
    train_Y = training['clarity'][train_index]
    test_X = X[test_index]
    test_Y = training['clarity'][test_index]
    xgb_clar.fit(train_X, train_Y)
    predictions = xgb_clar.predict(test_X)
    accuracy = accuracy_score(test_Y, predictions)
    mse = mean_squared_error(test_Y, predictions)
    rmse = math.sqrt(mse)
    avg_rmse += mse
    print("xgb model accuracy score for K fold {} = {}".format(step, accuracy) )
    print("xgb model rmse score for K fold {} = {}".format(step, rmse) )
    print()
    step += 1

avg_rmse = math.sqrt(avg_rmse/k)
print("AVG RMSE: ", avg_rmse)

xgb model accuracy score for K fold 1 = 0.9425196850393701
xgb model rmse score for K fold 1 = 0.23975052650751347

xgb model accuracy score for K fold 2 = 0.9440944881889763
xgb model rmse score for K fold 2 = 0.23644346430177263

xgb model accuracy score for K fold 3 = 0.9433070866141732
xgb model rmse score for K fold 3 = 0.2381027370397635

xgb model accuracy score for K fold 4 = 0.9405511811023622
xgb model rmse score for K fold 4 = 0.24382128475101963

xgb model accuracy score for K fold 5 = 0.9452755905511812
xgb model rmse score for K fold 5 = 0.23393248908353645

xgb model accuracy score for K fold 6 = 0.9440944881889763
xgb model rmse score for K fold 6 = 0.23644346430177263

xgb model accuracy score for K fold 7 = 0.9437007874015748
xgb model rmse score for K fold 7 = 0.23727455109729995

xgb model accuracy score for K fold 8 = 0.9397637795275591
xgb model rmse score for K fold 8 = 0.24543068364090287

xgb model accuracy score for K fold 9 = 0.9433070866141732
xgb model rmse

# **XGB PRIDICTION CONCISENESS**

In [72]:
    predictions = xgb_concise.predict(X_feats_val)
    accuracy = accuracy_score(testing['conciseness'], predictions)
    print("ACCURACY ON TESTING DATA VIA XGB FOR CONCISENESS",accuracy)


ACCURACY ON TESTING DATA VIA XGB FOR CONCISENESS 0.747840470501746


# **XGB PRIDICTION CLARITY**

In [73]:
    predictions = xgb_clar.predict(X_feats_val)
    accuracy = accuracy_score(testing['clarity'], predictions)
    print("ACCURACY ON TESTING DATA VIA XGB FOR CLARITY",accuracy)


ACCURACY ON TESTING DATA VIA XGB FOR CLARITY 0.9439441279176622
